<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [12]:
from bs4 import BeautifulSoup
import requests
import time, os

import sys
import importlib

# webdriver imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [7]:
# Configure window for multi-monitor automation
config_driver = webdriver.Chrome(chromedriver)

window_rect = (1300, 0, 500, 800) # x, y, width, height
config_driver.set_window_rect(*window_rect)

{'height': 800, 'width': 500, 'x': 1300, 'y': 23}

In [4]:
# End driver session after window is configured.
config_driver.quit()

In [11]:
def scroll_bottom(dr):
    '''
    
    Thwart infinite scroll. Takes driver as input
    Thanks to @ Matthew Eungoo Lee starting me off with this snippet.
    
    '''
    
    # Load Entire Page by Scrolling to the Bottom
    SCROLL_PAUSE_TIME = 2# Scroll to very Bottom to Load All
    last_height = dr.execute_script("return document.body.scrollHeight")# Get scroll height
    while True:    
        dr.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);"
        ) # Scroll down to bottom

        time.sleep(1) # Wait a sec

        # Go back up, to trigger infinite scroll
        dr.execute_script(
            "window.scrollTo(0, 0);"
        ) # Scroll down to bottom

        time.sleep(1) # Wait a sec

        # Then back down
        dr.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);"
        ) # Scroll down to bottom

        time.sleep(SCROLL_PAUSE_TIME) # Wait to load page

        new_height = dr.execute_script(
            "return document.body.scrollHeight"
        )# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:
            return True
        last_height = new_height

In [6]:
# Open the category page fully, in preparation for scrape.

test_category_url = 'https://castbox.fm/categories/10028?country=us'

def get_category_source(url, dr):
    '''
    Gets the source code of a category page using the scroll_bottom method.
    '''
    
    
#     dr.set_window_rect(*window_rect)
    dr.get(url)
    try:
        # close annoying cookie verification
        dr.find_element_by_class_name('allow').click()
    except:
        # sometimes the cookie doesn't appear
        pass
    scroll_bottom(dr)
    return dr.page_source

# driver = webdriver.Chrome(chromedriver)
# print(get_category_source(test_category_url, driver))

In [13]:
def scrape_full_category_page(url, dr):
    '''
    
    Crawl through the entirety of a category page.
    Return the scraped category listing,
    as well as the name of the category.
    
    '''
    
    
    # open page properly and get full source
    source = get_category_source(url, dr)
    time.sleep(0.2)

    # grab entire listing as dictionary
    soup = BeautifulSoup(source, 'lxml')
    category_dict = methods.collect_category_pods(soup)
    
    # page category
    category_name = soup.find(class_='guru-breadcrumb-item active').text
    
    return category_name, category_dict

In [10]:
driver = webdriver.Chrome(chromedriver)
scrape_full_category_page(test_category_url, driver).sample()

NameError: name 'scrape_full_category_page' is not defined

In [ ]:
business_html = requests.get(test_category_url).text
business_soup = BeautifulSoup(business_html)
business_soup.find(class_='guru-breadcrumb-item active').text

In [9]:
import pickle

In [10]:
# unpickle list of valid category directories
with open('../scraped/category/valid_category_url_list.pickle', 'rb') as file:
    valid_categories_url = pickle.load(file)

valid_categories_url[:5]

['https://castbox.fm/categories/10001?country=us',
 'https://castbox.fm/categories/10002?country=us',
 'https://castbox.fm/categories/10003?country=us',
 'https://castbox.fm/categories/10004?country=us',
 'https://castbox.fm/categories/10005?country=us']

In [15]:
# scrape all valid categories, save as dictionary

driver = webdriver.Chrome(chromedriver)
podcast_chan_dict_all_categories = {}

for i, cat_url in enumerate(valid_categories_url):
    cat_name, cat_dict = scrape_full_category_page(cat_url, driver, hide=True)
    key = (i, key, cat_url)
    podcast_chan_dict_all_categories[key] = cat_dict

driver.quit()

podcast_chan_dict_all_categories

FileNotFoundError: [Errno 2] No such file or directory: '../scraped/channel/podcast_chan_dict.pickle'

In [18]:
with open('../scraped/channel/podcast_chan_dict-v2.pickle','wb') as file:
    pickle.dump(podcast_chan_dict_all_categories, file)

file.close()

In [19]:
with open('../scraped/channel/podcast_chan_dict-v2.pickle','rb') as file:
    d = pickle.load(file)

d

{'Arts': {'Fresh Air': {'chan_url': 'https://castbox.fm//channel/Fresh-Air-id431951',
   'author': 'NPR'},
  'The Moth': {'chan_url': 'https://castbox.fm//channel/The-Moth-id12',
   'author': 'The Moth'},
  'TED Talks Daily': {'chan_url': 'https://castbox.fm//channel/TED-Talks-Daily-id4541',
   'author': 'TED'},
  '99% Invisible': {'chan_url': 'https://castbox.fm//channel/99%-Invisible-id18',
   'author': 'Roman Mars'},
  'Snap Judgment Presents: Spooked': {'chan_url': 'https://castbox.fm//channel/Snap-Judgment-Presents%3A-Spooked-id1022965',
   'author': 'Snap Judgment and WNYC Studios'},
  'Myths and Legends': {'chan_url': 'https://castbox.fm//channel/Myths-and-Legends-id227508',
   'author': 'Jason Weiser, Carissa Weiser / Bardic'},
  'Nice Try!': {'chan_url': 'https://castbox.fm//channel/Nice-Try!-id2112178',
   'author': 'Curbed'},
  'Snap Judgment': {'chan_url': 'https://castbox.fm//channel/Snap-Judgment-id131',
   'author': 'Snap Judgment and WNYC Studios'},
  'The Beauty Closet

In [28]:
# scraped dictionary should be of same length as valid category directory list:
print('number of valid categories: ', len(valid_categories_url))
print('keys in directory dictionary: ', len(podcast_chan_dict_all_categories))

number of valid categories:  126
keys in directory dictionary:  116


In [27]:
len(podcast_chan_dict_all_categories)

116

Hmm, looks like the scrape missed some keys in the category list. Well, thorough were the scrapes of each page, at least?

In [33]:
dict_lens = []
for key in podcast_chan_dict_all_categories.keys():
    dict_lens += [len(podcast_chan_dict_all_categories[key])]
    
print(dict_lens)

[200, 199, 200, 200, 200, 200, 60, 60, 60, 200, 60, 200, 200, 60, 60, 200, 200, 160, 200, 199, 200, 199, 200, 199, 200, 200, 200, 197, 199, 200, 95, 199, 198, 199, 199, 198, 157, 160, 200, 199, 61, 199, 150, 29, 200, 197, 90, 200, 200, 199, 200, 200, 200, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 59, 60, 60, 60, 60, 60, 60, 40, 10, 27, 60, 8, 60, 60, 17, 60, 60, 60, 60, 60, 60, 60, 59, 60, 25, 60, 60, 60, 60, 41, 60, 60, 12, 33, 8, 60, 60, 60, 60, 59, 60, 60, 60]


Some categories don't have up to the maximum of 200 entries. We can spot check the category URLs, and when we do we see that only some have 200 entries listed. Some have less, even as few as only 15 podcasts / category.

In [43]:
for i, val in enumerate(valid_categories_url[:20]):
    print(i, val)

0 https://castbox.fm/categories/10001?country=us
1 https://castbox.fm/categories/10002?country=us
2 https://castbox.fm/categories/10003?country=us
3 https://castbox.fm/categories/10004?country=us
4 https://castbox.fm/categories/10005?country=us
5 https://castbox.fm/categories/10006?country=us
6 https://castbox.fm/categories/10007?country=us
7 https://castbox.fm/categories/10008?country=us
8 https://castbox.fm/categories/10009?country=us
9 https://castbox.fm/categories/10010?country=us
10 https://castbox.fm/categories/10011?country=us
11 https://castbox.fm/categories/10012?country=us
12 https://castbox.fm/categories/10013?country=us
13 https://castbox.fm/categories/10014?country=us
14 https://castbox.fm/categories/10015?country=us
15 https://castbox.fm/categories/10016?country=us
16 https://castbox.fm/categories/10021?country=us
17 https://castbox.fm/categories/10022?country=us
18 https://castbox.fm/categories/10023?country=us
19 https://castbox.fm/categories/10024?country=us
